# 如何在无服务器环境中等待回调

:::info 预备知识

本指南假定您熟悉以下概念：

- [回调](/docs/concepts/callbacks)

:::

从 `@langchain/core@0.3.0` 开始，LangChain.js 的回调将在后台运行。这意味着执行将**不会**等待回调返回后再继续。在 `0.3.0` 版本之前，行为是相反的。

如果您在如 [AWS Lambda](https://aws.amazon.com/pm/lambda/) 或 [Cloudflare Workers](https://workers.cloudflare.com/) 等[无服务器环境](https://en.wikipedia.org/wiki/Serverless_computing)中运行代码，则应将回调设置为阻塞模式，以允许它们有时间完成或超时。

要使回调变为阻塞模式，请将环境变量 `LANGCHAIN_CALLBACKS_BACKGROUND` 设置为 `"false"`。或者，如有必要，您可以导入全局的 [`awaitAllCallbacks`](https://api.js.langchain.com/functions/langchain_core.callbacks_promises.awaitAllCallbacks.html) 方法以确保所有回调完成。

为了说明这一点，我们将创建一个需要一些时间才能完成的[自定义回调处理器](/docs/how_to/custom_callbacks)，并展示在未设置 `LANGCHAIN_CALLBACKS_BACKGROUND` 为 `"false"` 以及未使用 `awaitAllCallbacks` 全局方法时的时间表现：

In [2]:
import { RunnableLambda } from "@langchain/core/runnables";
import { awaitAllCallbacks } from "@langchain/core/callbacks/promises";

const runnable = RunnableLambda.from(() => "hello!");

const customHandler = {
  handleChainEnd: async () => {
    await new Promise((resolve) => setTimeout(resolve, 2000));
    console.log("Call finished");
  },
};

const startTime = new Date().getTime();

await runnable.invoke({ number: "2" }, { callbacks: [customHandler] });

console.log(`Elapsed time: ${new Date().getTime() - startTime}ms`);

await awaitAllCallbacks();

console.log(`Final elapsed time: ${new Date().getTime() - startTime}ms`);

Elapsed time: 1ms
Call finished
Final elapsed time: 2164ms


我们可以看到，初始的 `runnable.invoke()` 调用在短时间内完成，然后大约两秒后，回调完成。

以下是关闭后台执行后的效果：

In [3]:
process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "false";

const startTimeBlocking = new Date().getTime();

await runnable.invoke({ number: "2" }, { callbacks: [customHandler] });

console.log(`Initial elapsed time: ${new Date().getTime() - startTimeBlocking}ms`);

Call finished
Initial elapsed time: 2002ms


这一次，由于 `invoke()` 调用会等待回调返回后才返回，因此初始调用本身就需要两秒钟的时间。

## 后续步骤

现在，您已经了解了如何在后台运行回调以减少延迟。

接下来，请查看本部分中的其他操作指南，例如 [如何创建自定义回调处理程序](/docs/how_to/custom_callbacks)。